<a href="https://colab.research.google.com/github/JalesBussinguer/Remote_Sensing_Studies/blob/master/Desafio_Worcap_2020_91%2C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# Importação das bibliotecas básicas
import pandas as pd
import numpy as np

# Importação das bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Importação das bibliotecas e módulos de machine learning
from sklearn import ensemble, preprocessing, tree
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold
from yellowbrick.classifier import ConfusionMatrix, ROCAUC

In [2]:
# Importando os dados de treino (verdade terrestre)

df = "https://raw.githubusercontent.com/JalesBussinguer/Remote_Sensing_Studies/master/data/worcap_2020/dados_verdade_terrestre.csv"

dados = pd.read_csv(df)

dados.head()

,id,b1,b2,b3,b4,b5,b6,b7,b8,b9,pred_minus_obs_H_b1,pred_minus_obs_H_b2,pred_minus_obs_H_b3,pred_minus_obs_H_b4,pred_minus_obs_H_b5,pred_minus_obs_H_b6,pred_minus_obs_H_b7,pred_minus_obs_H_b8,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9,label
0,454,54,28,51,93,56,100,80,24,55,64.88,26.03,50.29,6.74,-30.83,-39.83,0.47,5.50,0.47,-24.52,-0.83,-3.99,-23.22,-0.94,-4.30,-23.22,-1.69,-4.55,s
1,457,54,29,51,88,51,91,95,24,55,56.28,20.62,42.67,10.16,-26.12,-32.05,-21.37,5.39,-0.30,-19.03,-0.54,-4.09,-22.24,-1.01,-4.93,-17.81,-0.53,-2.98,s
2,261,59,30,52,90,54,93,80,26,58,56.58,21.37,45.45,12.77,-28.17,-30.94,-1.30,3.64,-2.27,-21.04,-1.86,-5.61,-27.73,-1.15,-5.74,-21.81,-2.39,-5.07,s
3,8,53,27,49,95,49,92,63,25,54,66.97,24.43,49.28,8.08,-22.53,-28.25,19.78,3.75,0.92,-25.65,-2.09,-5.95,-39.27,-2.13,-8.73,-30.73,-2.42,-5.58,s
4,478,58,40,65,100,58,100,106,26,57,51.79,8.74,28.06,-1.00,-33.12,-41.04,-32.17,3.48,-2.08,-18.31,-0.20,-4.85,-21.15,-1.00,-4.84,-17.00,-0.91,-3.38,d


O conjunto de dados **`treino.csv`** será utilizado para construir os modelos de aprendizagem de máquina. Este apresenta a "verdade em terra" para cada classe por meio da coluna `label`.

In [3]:
# Importação dos dados de validação

dados_validacao = pd.read_csv("https://raw.githubusercontent.com/JalesBussinguer/Remote_Sensing_Studies/master/data/worcap_2020/dados_validacao.csv")

X_validacao = dados_validacao.drop(columns="id").values
dados_validacao

,id,b1,b2,b3,b4,b5,b6,b7,b8,b9,pred_minus_obs_H_b1,pred_minus_obs_H_b2,pred_minus_obs_H_b3,pred_minus_obs_H_b4,pred_minus_obs_H_b5,pred_minus_obs_H_b6,pred_minus_obs_H_b7,pred_minus_obs_H_b8,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9
0,356,79,28,54,117,52,99,93,25,60,34.87,24.16,45.04,-23.75,-26.66,-38.87,-18.61,4.12,-5.03,-18.51,-1.36,-4.23,-18.09,-0.70,-3.27,-15.79,-1.21,-3.39
1,202,46,27,50,83,51,90,76,26,56,68.54,24.27,48.21,16.37,-24.89,-27.66,2.19,4.93,1.25,-18.62,-2.17,-7.11,-21.12,-1.56,-6.35,-22.19,-4.45,-7.32
2,183,56,26,50,94,50,87,83,24,54,53.57,21.97,43.47,2.00,-25.22,-27.11,-8.04,5.69,1.43,-12.99,0.23,-5.97,-14.35,-0.55,-5.40,-14.51,-2.68,-5.15
3,306,56,58,81,69,65,100,109,42,80,56.41,-8.84,11.55,26.51,-40.14,-41.28,-32.79,-11.96,-24.36,-19.27,-0.81,-4.45,-18.37,-0.94,-4.23,-16.77,-1.35,-3.65
4,511,70,28,53,103,53,99,94,24,61,46.95,24.05,45.32,0.11,-28.10,-39.45,-14.22,5.33,-5.90,-23.26,-0.43,-4.16,-28.30,-1.13,-4.92,-23.06,-1.53,-4.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,7,40,39,58,82,61,99,89,26,57,73.99,12.91,41.92,17.33,-34.82,-36.19,-11.07,4.28,-0.19,-18.72,-2.61,-8.38,-20.56,-1.51,-6.68,-21.16,-3.42,-6.61
310,336,42,36,59,83,61,103,63,36,62,71.51,15.83,39.64,10.29,-35.53,-42.50,11.49,-7.01,-7.07,-19.28,-0.59,-1.60,-20.14,-0.36,-3.00,-15.86,-0.53,-2.80
311,345,32,32,53,84,77,116,61,35,70,82.01,19.24,44.45,11.09,-51.88,-56.35,13.48,-5.89,-15.05,-18.24,-0.89,-3.62,-19.14,-0.57,-3.16,-16.01,-1.36,-3.61
312,380,50,30,54,88,51,93,61,22,50,61.86,21.41,41.09,11.39,-25.61,-33.16,14.53,8.94,6.39,-17.50,-0.81,-2.66,-21.96,-0.76,-4.03,-19.09,-3.08,-5.35


O conjunto **`dados_validacao.csv`** será utilizado para realizar as predições das classes a partir do modelo preciamente treinado.

In [4]:
# Importação dos dados de verificação

ground_truth = pd.read_csv("https://raw.githubusercontent.com/JalesBussinguer/Remote_Sensing_Studies/master/data/worcap_2020/test_h.csv")

gt = ground_truth['label'].values
gt

array(['h ', 's ', 's ', 'o ', 'h ', 'd ', 'h ', 's ', 'h ', 'o ', 'd ',
       's ', 's ', 'd ', 'd ', 'o ', 's ', 'h ', 'o ', 's ', 'o ', 'd ',
       's ', 's ', 's ', 'd ', 'o ', 's ', 'd ', 's ', 'd ', 's ', 'd ',
       'o ', 'h ', 'h ', 's ', 'd ', 'd ', 'o ', 'h ', 's ', 'd ', 's ',
       'h ', 'o ', 's ', 'o ', 'o ', 'd ', 's ', 'd ', 'h ', 'h ', 'h ',
       's ', 'd ', 'd ', 's ', 'd ', 's ', 'h ', 's ', 's ', 'o ', 's ',
       'd ', 'h ', 's ', 's ', 'o ', 'h ', 'd ', 'd ', 'd ', 'h ', 'd ',
       'h ', 's ', 'h ', 'o ', 's ', 's ', 'h ', 'o ', 's ', 'd ', 'd ',
       'd ', 's ', 'h ', 'd ', 's ', 's ', 's ', 'h ', 'd ', 'd ', 'o ',
       'h ', 'd ', 'd ', 'h ', 's ', 'h ', 's ', 's ', 's ', 's ', 'd ',
       's ', 'o ', 'o ', 's ', 'd ', 'd ', 'o ', 's ', 'd ', 'h ', 'd ',
       's ', 's ', 's ', 'd ', 'o ', 'h ', 'h ', 's ', 's ', 'd ', 'h ',
       's ', 's ', 'd ', 'h ', 'h ', 'd ', 's ', 's ', 'd ', 's ', 'd ',
       'd ', 's ', 'o ', 'd ', 's ', 'd ', 'd ', 's

O conjunto **`test_h.csv`** será utilizado para avaliar a performance do modelo.

## Dicionário de variáveis:

* `id`: Identificação única da linha, alusiva a um pixel único das imagens;
* `b1`: Banda do verde corresponde ao mês de setembro de 2010;
* `b2`: Banda do vermelho corresponde ao mês de setembro de 2010;
* `b3`: Banda do infravermelho próximo corresponde ao mês de setembro de 2010;
* `b4`: Banda do verde corresponde ao mês de Março de 2011;
* `b5`: Banda do vermelho corresponde ao mês de Março de 2011;
* `b6`: Banda do infravermelho próximo corresponde ao mês de Março de 2011;
* `b7`: Banda do verde corresponde ao mês de Maio de 2011;
* `b8`: Banda do vermelho corresponde ao mês de Maio de 2011;
* `b9`: Banda do infravermelho próximo corresponde ao mês de Maio de 2011;
* `pred_minus_obs_S_b1` até `pred_minus_obs_S_b9`: Valores espectrais previstos (Com base na interpolação espacial) subtraídos dos valores espectrais reais para a classe "s";
* `pred_minus_obs_H_b1` até `pred_minus_obs_H_b9`: Valores espectrais previstos (Com base na interpolação espacial) subtraídos dos valores espectrais reais para a classe "h".

## Rótulos:

* `S` = *Sugi* (Floresta composta predominantemente pela espécie [*Cryptomeria japonica*](https://pt.wikipedia.org/wiki/Cryptomeria_japonica))
* `H` = *Hinoki* (Floresta composta predominantemente pela espécie [*Chamaecyparis obtusa*](https://en.wikipedia.org/wiki/Chamaecyparis_obtusa))
* `D` = *Mixed Deciduous* (Formação florestal composta por diversas espécies deciduais ou caducifólias);
* `O` = *Other* (Feição não-florestal)

Por se tratar de um problema de classificação de múltiplas classes, optou-se pela implementação de um modelo *Random Forest Classification*, pela sua facilidade de implementação.

# Análise Exploratória

Nesta primeira fase, é interessante verificar algumas características do dataset, para certificar que tudo está ok para a implementação do modelo de aprendizado de máquina.

## 1. Conferindo se existem dados faltando

In [5]:
# Expressão que conta os dados que faltam pelo tamanho do dataset, retornando a porcentagem faltante de dados

dados.isnull().sum()/dados.shape[0]

id                     0.0
b1                     0.0
b2                     0.0
b3                     0.0
b4                     0.0
b5                     0.0
b6                     0.0
b7                     0.0
b8                     0.0
b9                     0.0
pred_minus_obs_H_b1    0.0
pred_minus_obs_H_b2    0.0
pred_minus_obs_H_b3    0.0
pred_minus_obs_H_b4    0.0
pred_minus_obs_H_b5    0.0
pred_minus_obs_H_b6    0.0
pred_minus_obs_H_b7    0.0
pred_minus_obs_H_b8    0.0
pred_minus_obs_H_b9    0.0
pred_minus_obs_S_b1    0.0
pred_minus_obs_S_b2    0.0
pred_minus_obs_S_b3    0.0
pred_minus_obs_S_b4    0.0
pred_minus_obs_S_b5    0.0
pred_minus_obs_S_b6    0.0
pred_minus_obs_S_b7    0.0
pred_minus_obs_S_b8    0.0
pred_minus_obs_S_b9    0.0
label                  0.0
dtype: float64

No conjunto de dados de treino, não há dados faltantes. Portanto, podemos prosseguir na análise.

In [6]:
dados_validacao.isnull().sum()/dados_validacao.shape[0]

id                     0.0
b1                     0.0
b2                     0.0
b3                     0.0
b4                     0.0
b5                     0.0
b6                     0.0
b7                     0.0
b8                     0.0
b9                     0.0
pred_minus_obs_H_b1    0.0
pred_minus_obs_H_b2    0.0
pred_minus_obs_H_b3    0.0
pred_minus_obs_H_b4    0.0
pred_minus_obs_H_b5    0.0
pred_minus_obs_H_b6    0.0
pred_minus_obs_H_b7    0.0
pred_minus_obs_H_b8    0.0
pred_minus_obs_H_b9    0.0
pred_minus_obs_S_b1    0.0
pred_minus_obs_S_b2    0.0
pred_minus_obs_S_b3    0.0
pred_minus_obs_S_b4    0.0
pred_minus_obs_S_b5    0.0
pred_minus_obs_S_b6    0.0
pred_minus_obs_S_b7    0.0
pred_minus_obs_S_b8    0.0
pred_minus_obs_S_b9    0.0
dtype: float64

No conjunto de dados de teste também não existem dados faltantes. Portanto, prosseguimos para uma análise dos tipos de dados que temos.

In [7]:
# Verificação dos tipos de dados no conjunto de dados de treino

dados.dtypes

id                       int64
b1                       int64
b2                       int64
b3                       int64
b4                       int64
b5                       int64
b6                       int64
b7                       int64
b8                       int64
b9                       int64
pred_minus_obs_H_b1    float64
pred_minus_obs_H_b2    float64
pred_minus_obs_H_b3    float64
pred_minus_obs_H_b4    float64
pred_minus_obs_H_b5    float64
pred_minus_obs_H_b6    float64
pred_minus_obs_H_b7    float64
pred_minus_obs_H_b8    float64
pred_minus_obs_H_b9    float64
pred_minus_obs_S_b1    float64
pred_minus_obs_S_b2    float64
pred_minus_obs_S_b3    float64
pred_minus_obs_S_b4    float64
pred_minus_obs_S_b5    float64
pred_minus_obs_S_b6    float64
pred_minus_obs_S_b7    float64
pred_minus_obs_S_b8    float64
pred_minus_obs_S_b9    float64
label                   object
dtype: object

Observa-se que no conjunto de dados de treino, existem três tipos de dados distintos:
* Inteiros (int64): são os dados que representam os números digitais expressos nos pixels das imagens, e são diretamente relacionados à resposta espectral dos alvos da cena. Na coluna `id`, representam a identificação do pixel.
* Flutuantes (float64): são dados que representam o resultado de uma estatística que mede a diferença entre uma resposta espectral modelada e a resposta espectral observada;
* Texto (object): são os dados de rótulos dos pixels, representando a verdade terrestre das cenas.

In [8]:
# Verificação dos tipos de dados no conjunto de dados de treino

dados_validacao.dtypes

id                       int64
b1                       int64
b2                       int64
b3                       int64
b4                       int64
b5                       int64
b6                       int64
b7                       int64
b8                       int64
b9                       int64
pred_minus_obs_H_b1    float64
pred_minus_obs_H_b2    float64
pred_minus_obs_H_b3    float64
pred_minus_obs_H_b4    float64
pred_minus_obs_H_b5    float64
pred_minus_obs_H_b6    float64
pred_minus_obs_H_b7    float64
pred_minus_obs_H_b8    float64
pred_minus_obs_H_b9    float64
pred_minus_obs_S_b1    float64
pred_minus_obs_S_b2    float64
pred_minus_obs_S_b3    float64
pred_minus_obs_S_b4    float64
pred_minus_obs_S_b5    float64
pred_minus_obs_S_b6    float64
pred_minus_obs_S_b7    float64
pred_minus_obs_S_b8    float64
pred_minus_obs_S_b9    float64
dtype: object

O conjunto de dados de teste apresenta a mesma estrutura de tipos de dados que o conjunto de treino. Uma vez que os conjuntos de dados são compatíveis estruturalmente, podemos seguir para a verificação de valores únicos.

In [9]:
# Valores únicos (dados de treino)

dados.nunique()

id                     209
b1                      48
b2                      52
b3                      49
b4                      52
b5                      42
b6                      47
b7                      67
b8                      28
b9                      36
pred_minus_obs_H_b1    206
pred_minus_obs_H_b2    199
pred_minus_obs_H_b3    200
pred_minus_obs_H_b4    202
pred_minus_obs_H_b5    200
pred_minus_obs_H_b6    206
pred_minus_obs_H_b7    204
pred_minus_obs_H_b8    185
pred_minus_obs_H_b9    197
pred_minus_obs_S_b1    190
pred_minus_obs_S_b2    155
pred_minus_obs_S_b3    179
pred_minus_obs_S_b4    195
pred_minus_obs_S_b5    108
pred_minus_obs_S_b6    163
pred_minus_obs_S_b7    194
pred_minus_obs_S_b8    154
pred_minus_obs_S_b9    170
label                    4
dtype: int64

In [10]:
# Valores únicos (dados de teste)

dados_validacao.nunique()

id                     314
b1                      58
b2                      55
b3                      54
b4                      59
b5                      49
b6                      44
b7                      72
b8                      39
b9                      46
pred_minus_obs_H_b1    300
pred_minus_obs_H_b2    296
pred_minus_obs_H_b3    301
pred_minus_obs_H_b4    300
pred_minus_obs_H_b5    291
pred_minus_obs_H_b6    294
pred_minus_obs_H_b7    304
pred_minus_obs_H_b8    269
pred_minus_obs_H_b9    284
pred_minus_obs_S_b1    272
pred_minus_obs_S_b2    227
pred_minus_obs_S_b3    251
pred_minus_obs_S_b4    282
pred_minus_obs_S_b5    139
pred_minus_obs_S_b6    227
pred_minus_obs_S_b7    278
pred_minus_obs_S_b8    213
pred_minus_obs_S_b9    226
dtype: int64

# Construção do modelo:

## 1.  Pré-processamento:

In [11]:
# Remoção da coluna de identificação do conjunto de dados de treino

dados = dados.drop(columns="id")

In [12]:
# Separação dos atributos e dos rótulos

X = dados.iloc[:,:-1].values # atributos (amostra de pixels)
y = dados.iloc[:,-1].values # rótulos (verdade terreste)

In [13]:
# Informações importantes

print("Formato dos dados de treino:", "X =", X.shape, "e y =", y.shape)
print()
print("Formato dos dados de validação:", X_validacao.shape)
print("Formato dos dados de verdade terrestre:", gt.shape)
print()
print("Número de classes:", len(set(dados['label'].values)))
print("Número de features:", X.shape[1])

Formato dos dados de treino: X = (209, 27) e y = (209,)

Formato dos dados de validação: (314, 27)
Formato dos dados de verdade terrestre: (314,)

Número de classes: 4
Número de features: 27


Um indicativo de que tudo está ok é o fato do número de linhas ser igual para os conjuntos de dados de treino. O conjunto x possui 209 linhas (valores) e 27 colunas (features), enquanto o conjunto y possui 209 linhas (valores) e um única coluna com os rótulos.

Para o conjunto de teste, o indicativo de que está tudo certo é a quantidade de colunas (features), que bate com a quantidade de features do conjunto de treino.

# 3. Criando os modelos

## *Random Forest Classifier (default)*

In [48]:
rf_default = ensemble.RandomForestClassifier()

In [49]:
rf_default.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [52]:
conf_mat = confusion_matrix(gt, y_pred_default)
conf_mat

array([[ 90,   0,   1,   6],
       [  0,  48,   0,   5],
       [ 11,   0,  42,   0],
       [  3,   7,   0, 101]])

In [51]:
y_pred_default = rf_default.predict(X_validacao)
print("A acurácia do modelo default é:", round(precision_score(gt, y_pred_default, average='micro'), 3) * 100, "%")

A acurácia do modelo default é: 89.5 %


## Busca randomizada de otimizações para os hiperparâmetros do modelo (*randomized search*)

In [139]:
# Randomized Search

# n_estimators (quantidade de árvores na floresta)
n_estimators = [100, 250, 500, 750, 1000, 1500, 2500]
# random state
random_state = [37, 42]
# Criterion
criterion = ['gini', 'entropy']

# Dicionário de parâmetros a serem otimizados
rf_parametros = {'n_estimators': n_estimators,'criterion': criterion, 'random_state': random_state}

# Busca aleatória
RFtuning = GridSearchCV(rf_default, # modelo
                              param_grid=rf_parametros, # parâmetros
                              cv=KFold(n_splits=10), # validação cruzada
                              scoring= 'accuracy', ## métrica de ajuste
                              n_jobs=-1, verbose =1).fit(X, y)

print(f'Melhor resultado: {RFtuning.best_score_} para {RFtuning.best_params_}')

Fitting 10 folds for each of 28 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  6.5min finished


Melhor resultado: 0.8704761904761904 para {'criterion': 'entropy', 'n_estimators': 500, 'random_state': 37}


## Modelagem final

In [145]:
modelo_final = ensemble.RandomForestClassifier(n_estimators=750, criterion='entropy', random_state=37, n_jobs=-1)

In [146]:
modelo_final.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=750,
                       n_jobs=-1, oob_score=False, random_state=37, verbose=0,
                       warm_start=False)

In [147]:
y_verificacao = modelo_final.predict(X_validacao)

In [148]:
conf_mat = confusion_matrix(gt, y_verificacao)
conf_mat

array([[ 92,   0,   1,   4],
       [  0,  47,   0,   6],
       [ 10,   0,  43,   0],
       [  2,   5,   0, 104]])

In [149]:
print("A precisão do modelo otimizado é:", round(precision_score(gt, y_verificacao, average='micro'), 5) * 100, "%")

A precisão do modelo otimizado é: 91.083 %


In [150]:
f1_score(gt, y_verificacao, average='micro')

0.910828025477707

In [151]:
recall_score(gt, y_verificacao, average='micro')

0.910828025477707